In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.nn import Module
from pprint import pprint
import copy
import torch

from sentence_transformers import SentenceTransformer
st_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


class inspect_func(Module):
    def __call__(self, *args, **kwargs):
        pprint(args)
        pprint(kwargs)
        print(kwargs["encoder_hidden_states"].shape)


In [70]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def concat_dicts(dict_1, dict_2):
    for k in dict_1:
        dict_1[k] = torch.cat((dict_1[k], dict_2[k]),dim=1)
    return dict_1



class STOnlyEncoder(Module):
    def __init_(self, encoder, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.st_encoder = encoder

    def forward(self, input_ids, **kwargs):
        


class CustomEncoder(Module):
    def __init__(self, encoder, *args, **kwargs):    
        super().__init__(*args, **kwargs)
        self.t5_encoder = encoder

    def forward(self, input_ids, **kwargs):
        print("kwargs")
        print(kwargs)

        input_ids = input_ids[:input_ids.shape[0],:input_ids.shape[1]//2]
        kwargs["attention_mask"] = kwargs["attention_mask"][:input_ids.shape[0], :input_ids.shape[1]]
        encoder_1_output = self.t5_encoder.forward(input_ids, **kwargs)
        encoder_2_output = self.t5_encoder.forward(input_ids, **kwargs)
        print("input ids shape",input_ids.shape)
        print("attention mask shape", kwargs["attention_mask"].shape)
        print(encoder_1_output["last_hidden_state"].shape)

        encoder_output = concat_dicts(encoder_1_output, encoder_2_output)
        print(dir(encoder_1_output))
        #print(encoder_output)
        print(encoder_output["last_hidden_state"].shape)

        return(encoder_output)



In [71]:
#class CustomT5(T5ForConditionalGeneration):
#    def __init__(self, *args, **kwargs):    
#        super().__init__(*args, **kwargs)
#        #self.t5_tokenizer = tokenizer
    
#    def from_pretrained(*args, **kwargs):
#        print(*args)
#        custom_model = T5ForConditionalGeneration.from_pretrained(*args, **kwargs)


#    def set_encoder(self):
#        self.encoder = super().encoder

                


In [68]:
from IPython.display import clear_output
model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

model.encoder = CustomEncoder(model.encoder)

clear_output()

In [69]:
tokens = tokenizer.encode("Translate to German: This is an unreasonably long sentence", return_tensors="pt")
print(tokens)
tokens = torch.cat((tokens, tokens), dim=1)
generated = model.generate(tokens)
tokenizer.decode(generated[0], skip_special_tokens=True)

tensor([[30355,    15,    12,  2968,    10,   100,    19,    46,    73,   864,
           739,  3834,   307,  7142,     1]])
kwargs
{'return_dict': True, 'output_attentions': False, 'output_hidden_states': False, 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}


RuntimeError: The size of tensor a (15) must match the size of tensor b (30) at non-singleton dimension 3

In [60]:
tokens

tensor([[30355,    15,    12,  2968,    10,   100,    19,     3,     9, 13145,
           307,  7142,     1, 30355,    15,    12,  2968,    10,   100,    19,
             3,     9, 13145,   307,  7142,     1]])

In [61]:
model.forward(tokens)

kwargs
{'attention_mask': None, 'inputs_embeds': None, 'head_mask': None, 'output_attentions': None, 'output_hidden_states': None, 'return_dict': True}


TypeError: 'NoneType' object is not subscriptable

In [ ]:
t5_generated = t5_model.generate(tokens)
tokenizer.decode(t5_generated[0], skip_special_tokens=True)

In [ ]:
generated = model.generate(tokens)
tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
t5_model.forward(tokens)